In [ ]:
import pandas as pd
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import statsmodels.api as sm
import matplotlib
from pylab import rcParams

In [ ]:
# Read the data file
df =

In [ ]:
df.head()

### Analysing Office Supply category
There are three categories of sales: Furniture, Office Supply and Technology.
Work on time series analysis and forecasting for office Supply sales.

In [ ]:
df_office =
df_office

In [ ]:
# Lets look at the date range of sales of data


In [ ]:
# Check for null values, we focus on order date and sales column and make sure there are no missing values.


In [ ]:
columns = ['Row ID', 'Order ID', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Quantity', 'Discount', 'Profit']
df_office.drop(columns, axis=1, inplace=True)

In [ ]:
df_office.head()

In [ ]:
# Set date as the index refer to previous lab


In [ ]:
# Plot the sales data


In [30]:
# Since it's really hard to read the sales data in current format,resample the data on Monthly basis


In [ ]:
# Plot the sales data based on monthly basis


### Time Decomposition

Distinctive patterns emerge when visualizing the data. The time series exhibits a seasonal pattern, with sales consistently starting low at the beginning of each year and peaking at the year's end. Within each individual year, there is a consistent upward trend, interspersed with a few months of lower sales in the middle of the year.

Time Series decomposition allows us to decompose our time series into three distinct components: trend, seasonality, and noise.

In essence, time decomposition is like taking apart a puzzle to see its individual pieces. It helps us grasp the different elements that contribute to the overall pattern of the time series, leading to a more informed understanding and better decision-making.

In [ ]:
# Find the different components like trean, seasonal


### Check for Stationarity

This is a very important concept in Time Series Analysis. In order to apply a time series model, it is important for the Time series to be stationary; in other words all its statistical properties (mean,variance) remain constant over time.

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):

    #Determing rolling statistics
    rolmean = timeseries.rolling(window=12).mean()
    rolstd = timeseries.rolling(window=12).std()
    #Plot rolling statistics:
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
# Using above function, pass you time series data, check if the p-value is less than 0.05



## Time series forecasting with ARIMA

ARIMA models are denoted with the notation ARIMA(p, d, q). These three parameters account for seasonality, trend, and noise in data.

In [ ]:
p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
for param in pdq:
    for param_seasonal in seasonal_pdq:

      try:
          mod = sm.tsa.statespace.SARIMAX( ,   # pass you timeseries datafrmae
                                          order=param,
                                          seasonal_order=param_seasonal,
                                          enforce_stationarity=False,
                                          enforce_invertibility=False)
          results = mod.fit()

          print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
      except:
          continue

#### From Above output check which AIC yields the lowest value.



## Fitting the ARIMA model

In [ ]:
# Use the AIC values and pass them in order and seasonal order
mod = sm.tsa.statespace.SARIMAX(df_office,
                                order= ,
                                seasonal_order= ,
                                enforce_stationarity=False,
                                enforce_invertibility=False)
results = mod.fit()
print(results.summary().tables[1])

In [ ]:
# Predict the office supplies for year 2014
pred = results.get_prediction(start=pd.to_datetime(' '), dynamic=False)
pred_ci = pred.conf_int()

#pass you TS dataframe
ax = Pass_TS_Dataframe['2011':].plot(label='observed')

pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 7))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Furniture Sales')
plt.legend()
plt.show()